<img src="https://i.imgur.com/lemJ6qf.png" align=center> 

# CMEMS_2 SAIL ARIANE / OPENDRIFT 
## SAIL THROUGH THE INDONESIAN THROUGHFLOW with OpenDrift, a lagrangian particles trajectories tool

OpenDrift is a software package for modeling the trajectories and fate of objects or substances drifting in the ocean, or even in the atmosphere.
The aim of this exercise is to follow the trajectories of particules sed in the Indonesian throughflow over a period to study where they are likely to go and land. We will use the Global Ocean Physics Analysis and Forecast product to do so.

***
**General Note 1**: Execute each cell through the <button class="btn btn-default btn-xs"><i class="icon-play fa fa-play"></i></button> button from the top MENU (or keyboard shortcut `Shift` + `Enter`).<br>
<br>
**General Note 2**: If, for any reason, the kernel is not working anymore, in the top MENU, click on the <button class="btn btn-default btn-xs"><i class="fa fa-repeat icon-repeat"></i></button> button. Then, in the top MENU, click on "Run" and select "Run All Above Selected Cell".<br><br>
**General Note 3**: Download the file with all dependencies and unzip while preserving file structure in the same directory as the notebook: [Download dependency file ARIANE](https://atlas.mercator-ocean.fr/s/xLrmSTgymBgAirs)<br>
***

First, the notebook must be set up with all the necessary tools available from the Jupyter Notebook Ecosystem.

| Module name | Description |
| :---: | :---|
| **datetime** | [Datetime](https://docs.python.org/fr/3/library/datetime.html) allows managing dates and times variables |
| **numpy** | [NumPy](https://numpy.org/) allows carrying out scientific computing with Python and managing ND-arrays |
| **xarray** | [Xarray](http://xarray.pydata.org/en/stable/) handles netCDF files in an intuitive and interactive way. |
| **matplotlib** |[Matplotlib](https://matplotlib.org/) is a Python numerical plotting library |
| **cartopy** |[Cartopy](https://scitools.org.uk/cartopy/docs/latest/) is a library for plotting 2D data on maps. |
| **opendrift** |[OpenDrift](https://opendrift.github.io/) models the trajectories of objects drifting in the ocean|

In [3]:
from datetime import datetime, timedelta
import numpy as np 
from opendrift.readers import reader_netCDF_CF_generic
from opendrift.readers import reader_shape
from opendrift.models.oceandrift import OceanDrift
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import xarray as xr

The input file has already been prepared for the exercise. It contains the current variables (x and y direction)for the period we intend to study : June to August 2015. Let's have a look at it.

In [2]:
path_file = 'ARIANE/DATA/glorys12v1_landmask_v3.nc'
file = xr.open_dataset(path_file)
print(file)

<xarray.Dataset>
Dimensions:               (depth: 1, latitude: 481, longitude: 841, time: 61)
Coordinates:
  * depth                 (depth) float32 0.494
  * latitude              (latitude) float32 -20.0 -19.92 -19.83 ... 19.92 20.0
  * longitude             (longitude) float32 90.0 90.08 90.17 ... 159.9 160.0
  * time                  (time) datetime64[ns] 2015-06-01T12:00:00 ... 2015-...
Data variables:
    land_binary_mask      (time, latitude, longitude) float64 ...
    x_sea_water_velocity  (time, depth, latitude, longitude) float32 ...
    y_sea_water_velocity  (time, depth, latitude, longitude) float32 ...
Attributes: (12/26)
    CDI:                                Climate Data Interface version 1.9.8 ...
    Conventions:                        CF-1.4
    history:                            Thu Apr  1 17:49:14 2021: ncks -x -v ...
    source:                             MERCATOR GLORYS12V1
    institution:                        MERCATOR OCEAN
    title:                      

We will now initialize OpenDrift  with this file : 

In [3]:
o = OceanDrift(loglevel=0)  # Set loglevel to 0 for debug information

reader_analysis = reader_netCDF_CF_generic.Reader("ARIANE/DATA/glorys12v1_landmask_v3.nc")

shpfilename = shpreader.natural_earth(resolution='110m',
                category='cultural',
                name='admin_0_countries')
reader_natural = reader_shape.Reader.from_shpfiles(shpfilename)

o.add_reader([reader_natural, reader_analysis])
o.set_config('general:use_auto_landmask', False)  # Disabling the automatic GSHHG landmask
o.set_config('general:coastline_action', 'stranding')
print("reader init ok")

08:09:29 DEBUG   opendrift.models.basemodel: Adding 17 config items from basemodel
08:09:29 DEBUG   opendrift.models.basemodel: Adding 4 config items from basemodel
08:09:29 DEBUG   opendrift.models.basemodel: Adding 34 config items from basemodel
08:09:29 INFO    opendrift.models.basemodel: OpenDriftSimulation initialised (version 1.5.6)
08:09:29 DEBUG   opendrift.models.basemodel: Adding 13 config items from oceandrift
08:09:29 DEBUG   opendrift.models.basemodel:   Overwriting config item seed:z
08:09:29 INFO    opendrift.readers.reader_netCDF_CF_generic: Opening dataset: ARIANE/DATA/glorys12v1_landmask_v3.nc
08:09:29 DEBUG   opendrift.readers.reader_netCDF_CF_generic: Finding coordinate variables.
08:09:29 INFO    opendrift.readers.reader_netCDF_CF_generic: Grid coordinates are detected, but proj4 string not given: assuming latlong
08:09:29 DEBUG   opendrift.readers.basereader.variables: Setting buffer size 49 for reader ARIANE/DATA/glorys12v1_landmask_v3.nc, assuming a maximum aver

reader init ok


We now initialize the particules. You can adapt the following cell to define the location of the particules to seed. Change the variables lon/lat min/max and nb_points to define the geographical extent and density of particules. With another input file, you may also change the starting date. 

In [4]:
seed_time = datetime(2015,6,1)

lons=[]
lats=[]

lon_min = 125
lon_max = 130
lat_min = -5
lat_max = 0

nb_points = 60
dx = (lon_max-lon_min)/nb_points
dy = (lat_max-lat_min)/nb_points

for i in range(nb_points):
        for j in range(nb_points):
                lons.append(lon_min+i*dx)
                lats.append(lat_min+j*dy)

time_step = timedelta(days=1)
o.seed_elements(lons, lats, time=seed_time)

We can now run OpenDrift. The following cell returns a netcdf and jpeg file you can access in *ARIANE/RES* directory. Caution : the output of the cell is rather long as OpenDrift reports each task.

In [5]:
print(o)

print("launch run")

o.run(steps=61, time_step=3600*24,outfile="ARIANE/RES/OpenDrift_res_06_08_2015.nc")

print("run finished")
o.plot(filename="ARIANE/RES/OpenDrift_res_06_08_2015.png")

Model:	OceanDrift     (OpenDrift version 1.5.6)
	0 active Lagrangian3DArray particles  (0 deactivated, 3600 scheduled)
-------------------
Environment variables:
  -----
  land_binary_mask
     1) shape
     2) ARIANE/DATA/glorys12v1_landmask_v3.nc
  -----
  x_sea_water_velocity
  y_sea_water_velocity
     1) ARIANE/DATA/glorys12v1_landmask_v3.nc
  -----
Readers not added for the following variables:
  ocean_vertical_diffusivity
  sea_floor_depth_below_sea_level
  sea_surface_wave_mean_period_from_variance_spectral_density_second_frequency_moment
  sea_surface_wave_period_at_variance_spectral_density_maximum
  sea_surface_wave_significant_height
  sea_surface_wave_stokes_drift_x_velocity
  sea_surface_wave_stokes_drift_y_velocity
  surface_downward_x_stress
  surface_downward_y_stress
  turbulent_generic_length_scale
  turbulent_kinetic_energy
  upward_sea_water_velocity
  x_wind
  y_wind

launch run


08:09:36 DEBUG   opendrift.models.basemodel: 
------------------------------------------------------
Software and hardware:
  OpenDrift version 1.5.6
  Platform: Linux, 4.19-ovh-xxxx-std-ipv6-64
  125.76339721679688 GB memory
  48 processors (x86_64)
  NumPy version 1.20.2
  SciPy version 1.6.2
  Matplotlib version 3.4.1
  NetCDF4 version 1.5.6
  Xarray version 0.17.0
  OilLibrary version 1.1.3
  Python version 3.9.2 | packaged by conda-forge | (default, Feb 21 2021, 05:02:46) [GCC 9.3.0]
------------------------------------------------------

08:09:36 INFO    opendrift.models.basemodel: Fallback values will be used for the following variables which have no readers: 
08:09:36 INFO    opendrift.models.basemodel: 	x_wind: 0.000000
08:09:36 INFO    opendrift.models.basemodel: 	y_wind: 0.000000
08:09:36 INFO    opendrift.models.basemodel: 	upward_sea_water_velocity: 0.000000
08:09:36 INFO    opendrift.models.basemodel: 	ocean_vertical_diffusivity: 0.000000
08:09:36 INFO    opendrift.models

run finished


08:09:56 INFO    opendrift.models.basemodel: Time to make plot: 0:00:11.536662


(<GeoAxesSubplot:title={'center':'OpenDrift - OceanDrift\n2015-06-01 00:00 to 2015-08-01 00:00 UTC (62 steps)'}>,
 <module 'matplotlib.pyplot' from '/opt/anaconda3/envs/opendrift/lib/python3.9/site-packages/matplotlib/pyplot.py'>)

<img src="https://i.imgur.com/7DEE3MT.png" align=center> 

You can see in the map above the trajectory of the particules, the coasts where some have landed and where the other ones are drifting.

In [4]:
path_file1 = 'ARIANE/RES/OpenDrift_res_06_08_2015.nc'
file1 = xr.open_dataset(path_file1)
print(file1)

<xarray.Dataset>
Dimensions:                                                                              (time: 62, trajectory: 3600)
Coordinates:
  * trajectory                                                                           (trajectory) int32 ...
  * time                                                                                 (time) datetime64[ns] ...
    lon                                                                                  (trajectory, time) float32 ...
    lat                                                                                  (trajectory, time) float32 ...
Data variables: (12/24)
    status                                                                               (trajectory, time) int32 ...
    moving                                                                               (trajectory, time) int32 ...
    age_seconds                                                                          (trajectory, time) float32 ...
    o

In [5]:
dates = file1['time'].data
for i in range(len(dates)):
    data_crs = ccrs.PlateCarree()
    projection=ccrs.PlateCarree(central_longitude=0)
    f = plt.figure(figsize=(12, 12))
    ax = plt.axes(projection=projection)
    
    ax.coastlines()
    ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
    ax.add_feature(cfeature.LAND, zorder=100, edgecolor='k')
    ax.set_extent([90, 160, -20, 20],crs=ccrs.PlateCarree())
    
    ax.scatter(file1['lon'][:,i],file1['lat'][:,i],c='tab:red',s=1)
    
    ax.set_title('OpenDrift simulation - '+ np.datetime_as_string(dates[i], unit='D'),fontsize=24)
    plt.savefig('ARIANE/FIGURES/OpenDrift_trajectory_'+str(i).zfill(3)+'.png')